## See also _last_exam: LINQ + extension method is the principle

In [3]:

public static IEnumerable<char> Characters(this string input)
{
	foreach (var ch in input)
	{
		yield return ch;
	}
}

var test = "Hello!".Characters();
foreach(var c in test)
{
	Console.WriteLine(c);
}

// enumerator stuff ... see yield-return
IEnumerator<char> enumerator = test.GetEnumerator();

H
e
l
l
o
!


In [4]:
var test_filtered = from c in test where Char.IsLetter(c) select Char.ToUpper(c);

foreach(var c in test_filtered)
{
    Console.WriteLine(c);
}

H
E
L
L
O


### Generating (Mapping to) Objects

In [8]:
public class Person
{
    public char FavoriteLetter { get; set; }
    public string Name { get; set; }
    public int Age { get; set; }
}

var test_people = from c in test_filtered select new Person { FavoriteLetter = c, Name = "John", Age = 30 };

foreach(var p in test_people)
{
    Console.WriteLine(p.Name + " " + p.Age + " " + p.FavoriteLetter);
} 

John 30 H
John 30 E
John 30 L
John 30 L
John 30 O
